In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Rheumatoid_Arthritis"
cohort = "GSE186963"

# Input paths
in_trait_dir = "../../input/GEO/Rheumatoid_Arthritis"
in_cohort_dir = "../../input/GEO/Rheumatoid_Arthritis/GSE186963"

# Output paths
out_data_file = "../../output/preprocess/Rheumatoid_Arthritis/GSE186963.csv"
out_gene_data_file = "../../output/preprocess/Rheumatoid_Arthritis/gene_data/GSE186963.csv"
out_clinical_data_file = "../../output/preprocess/Rheumatoid_Arthritis/clinical_data/GSE186963.csv"
json_path = "../../output/preprocess/Rheumatoid_Arthritis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Whole blood gene expression from infliximab treated Crohn's disease patients at three time points: pre-treatment, two weeks and fourteen weeks post first treatment"
!Series_summary	"Personalized treatment of complex diseases is an unmet medical need pushing towards drug biomarker identification of one drug-disease combination at a time. Here, we used a novel computational approach for modeling cell-centered individual-level network dynamics from high-dimensional blood data to predict infliximab response and uncover individual variation of non-response. We identified and validated that the RAC1-PAK1 axis is predictive of infliximab response in inflammatory bowel disease. Intermediate monocytes, which closely correlated with inflammation state, play a key role in the RAC1-PAK1 responses, supporting their modulation as a therapeutic target. This axis also predicts response in Rheumatoid arthritis, validated in three public cohorts. Our findings suppo

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import re
from typing import Optional, Callable, Any, Dict

# 1. Gene Expression Data Availability
# Yes, the dataset likely contains gene expression data as it mentions "Whole blood gene expression"
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (Rheumatoid Arthritis):
# From the background info, this dataset is for Crohn's disease, not RA
trait_row = None

# For age:
# Age is not mentioned in the sample characteristics dictionary
age_row = None

# For gender:
# Gender is not mentioned in the sample characteristics dictionary
gender_row = None

# 2.2 Data Type Conversion
# Even though we don't have trait data for Rheumatoid Arthritis,
# define conversion functions in case they're needed

def convert_trait(value):
    # Extract value after colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # This dataset doesn't have RA trait, but if it did:
    if value.lower() in ['rheumatoid arthritis', 'ra', 'true', 'yes']:
        return 1
    elif value.lower() in ['control', 'healthy', 'false', 'no']:
        return 0
    return None

def convert_age(value):
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    value = value.lower()
    if value in ['female', 'f', 'woman']:
        return 0
    elif value in ['male', 'm', 'man']:
        return 1
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this substep
# (if trait_row were not None, we would extract clinical features)


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1',
       'TC0100006480.hg.1', 'TC0100006483.hg.1', 'TC0100006486.hg.1',
       'TC0100006490.hg.1', 'TC0100006492.hg.1', 'TC0100006494.hg.1',
       'TC0100006497.hg.1', 'TC0100006499.hg.1', 'TC0100006501.hg.1',
       'TC0100006502.hg.1', 'TC0100006514.hg.1', 'TC0100006516.hg.1',
       'TC0100006517.hg.1', 'TC0100006524.hg.1', 'TC0100006540.hg.1',
       'TC0100006548.hg.1', 'TC0100006550.hg.1'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These gene identifiers appear to be probe IDs from a microarray platform, likely Affymetrix 
# or similar custom array. They follow a format like "TC0100006437.hg.1" which indicates 
# they are transcript cluster IDs with a human genome reference (.hg).
# These are definitely not standard human gene symbols (like BRCA1, TP53, etc.).
# They will need to be mapped to standard gene symbols for meaningful biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'probeset_id': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['69091', '924880', '960587', '966497', '1001138'], 'stop': ['70008', '944581', '965719', '975865', '1014541'], 'total_probes': [10.0, 10.0, 10.0, 10.0, 10.0], 'category': ['main', 'main', 'main', 'main', 'main'], 'SPOT_ID': ['Coding', 'Multiple_Complex', 'Multiple_Complex', 'Multiple_Complex', 'Multiple_Complex'], 'SPOT_ID.1': ['NM_001005484 // RefSeq // Homo sapiens olfactory receptor, family 4, subfamily F, member 5 (OR4F5), mRNA. // chr1 // 100 // 100 // 0 // --- // 0 /// ENST00000335137 // ENSEMBL // olfactory receptor, family 4, subfamily F, member 5 [gene_biotype:protein_coding transcript_biotype:protein_coding] // chr1 // 10

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns contain gene identifiers and gene symbols
# 'ID' in gene_annotation matches the probe IDs in gene_data.index
# 'SPOT_ID.1' contains the gene information with gene symbols

# 2. Create a gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'SPOT_ID.1')

# Check the mapping dataframe content
print(f"Number of mappings before filtering: {len(mapping_df)}")
print(f"Sample of mapping dataframe (first 3 rows):")
print(mapping_df.head(3))

# 3. Convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print info before normalization
print(f"Before normalization - number of genes: {len(gene_data)}")
if len(gene_data) > 0:
    print(f"Before normalization - first 5 genes: {gene_data.index[:5]}")
else:
    print("No genes were mapped successfully before normalization.")

# Normalize gene symbols to ensure consistency
gene_data = normalize_gene_symbols_in_index(gene_data)

# Print info after normalization
print(f"After normalization - number of genes: {len(gene_data)}")
if len(gene_data) > 0:
    print(f"After normalization - first 5 genes: {gene_data.index[:5]}")
else:
    print("No genes were mapped successfully after normalization.")


Number of mappings before filtering: 27189
Sample of mapping dataframe (first 3 rows):
                  ID                                               Gene
0  TC0100006437.hg.1  NM_001005484 // RefSeq // Homo sapiens olfacto...
1  TC0100006476.hg.1  NM_152486 // RefSeq // Homo sapiens sterile al...
2  TC0100006479.hg.1  NM_198317 // RefSeq // Homo sapiens kelch-like...


Before normalization - number of genes: 85192
Before normalization - first 5 genes: Index(['A-', 'A-1', 'A-2', 'A-52', 'A-E'], dtype='object', name='Gene')
After normalization - number of genes: 19964
After normalization - first 5 genes: Index(['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2'], dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. We cannot process clinical features since trait_row is None (no RA data in this dataset)
# This means we're skipping the clinical feature extraction step as specified in the instructions

# Save the gene data that we've already normalized
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Since there's no trait data in this dataset, we cannot link clinical and genetic data
# We'll create empty placeholder data to properly finish the process
print("No trait data available for linking with gene expression data.")

# Create a small placeholder dataframe with proper structure
empty_clinical_df = pd.DataFrame(columns=[trait])
# Use first few samples from gene_data for consistent structure
sample_ids = gene_data.columns[:5] if len(gene_data.columns) >= 5 else gene_data.columns
empty_clinical_df = pd.DataFrame(index=sample_ids, columns=[trait])
empty_clinical_df[trait] = None  # All None values for trait

# Skip handling missing values since we don't have valid linked data

# 5. We'll mark this dataset as not usable due to missing trait data
is_trait_biased = True  # Not actually assessed, but marked as biased since trait data is missing
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=False,  # No trait data
    is_biased=is_trait_biased, 
    df=empty_clinical_df,
    note="This dataset contains gene expression data for Crohn's disease, not Rheumatoid Arthritis"
)

# 6. Not saving linked data as it's not usable for our purposes
print("No valid linked data to save - dataset doesn't contain Rheumatoid Arthritis data")

Normalized gene data saved to ../../output/preprocess/Rheumatoid_Arthritis/gene_data/GSE186963.csv
No trait data available for linking with gene expression data.
Abnormality detected in the cohort: GSE186963. Preprocessing failed.
No valid linked data to save - dataset doesn't contain Rheumatoid Arthritis data
